## Imports

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
os.environ['PYOPENGL_PLATFORM'] = 'osmesa'
#os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [ ]:
import numpy as np
import trimesh
import matplotlib.pyplot as plt
import json
from datareader import Ho3dReader
from run_attachment import MeshWithConfidence

In [ ]:
import pyrender

## Functions

In [ ]:
# Coordinate system transformation (OpenCV cam to OpenGL cam)
cvcam_in_glcam = np.array([[1, 0, 0, 0],
                          [0, -1, 0, 0],
                          [0, 0, -1, 0],
                          [0, 0, 0, 1]])

def render_mesh_at_pose(mesh, pose, K, H, W, zfar=100):
    # Transform mesh to OpenGL camera frame
    mesh_transformed = mesh.copy()
    mesh_transformed.apply_transform(cvcam_in_glcam @ pose)
    
    # Setup pyrender scene
    scene = pyrender.Scene(ambient_light=[1., 1., 1.], bg_color=[0, 0, 0])
    
    # Add camera
    camera = pyrender.IntrinsicsCamera(
        fx=K[0, 0], fy=K[1, 1],
        cx=K[0, 2], cy=K[1, 2],
        znear=0.1, zfar=zfar
    )
    scene.add(camera, pose=np.eye(4))
    
    # Add mesh
    pyrender_mesh = pyrender.Mesh.from_trimesh(mesh_transformed, smooth=False)
    scene.add(pyrender_mesh, pose=np.eye(4))
    
    # Render
    renderer = pyrender.OffscreenRenderer(W, H)
    color, depth = renderer.render(scene)
    renderer.delete()
    
    return color, depth

def visualize_comparison(mesh, pose, rgb, K):    
    # Check if mesh is a file path or trimesh object
    if isinstance(mesh, str):
        mesh = trimesh.load(mesh)

    H, W = rgb.shape[:2]
    # Render mesh at gt_pose
    rendered, depth = render_mesh_at_pose(mesh, pose, K, H, W)
    
    # Create visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(rgb)
    axes[0].set_title('Original RGB Image')
    axes[0].axis('off')
    
    # Rendered mesh
    axes[1].imshow(rendered)
    axes[1].set_title('Rendered Mesh at GT Pose')
    axes[1].axis('off')
    
    # Overlay
    overlay = rgb.copy()
    mask = rendered.sum(axis=2) > 0
    overlay[mask] = rendered[mask] * 0.6 + rgb[mask] * 0.4
    axes[2].imshow(overlay)
    axes[2].set_title('Overlay (60% mesh, 40% RGB)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_confidence_pointcloud(CMesh):
    """Visualize mesh vertices as colored point cloud by confidence
    
    Colors: red (0) → orange (0-0.25) → yellow (0.25-0.5) → blue (0.5-1) → green (1)
    """
    import plotly.graph_objects as go
    
    # Assign colors based on confidence
    colors = np.zeros((len(CMesh.confidence), 3), dtype=np.uint8)
    
    mask_zero = CMesh.confidence == 0
    mask_low = (CMesh.confidence > 0) & (CMesh.confidence < 0.25)
    mask_mid = (CMesh.confidence >= 0.25) & (CMesh.confidence < 0.5)
    mask_high = (CMesh.confidence >= 0.5) & (CMesh.confidence < 1)
    mask_one = CMesh.confidence >= 1.0
    
    colors[mask_zero] = [255, 0, 0]      # Red
    colors[mask_low] = [255, 125, 0]     # Orange
    colors[mask_mid] = [255, 255, 0]     # Yellow
    colors[mask_high] = [0, 0, 255]      # Blue
    colors[mask_one] = [0, 255, 0]       # Green
    
    # Create plotly scatter3d
    fig = go.Figure(data=[go.Scatter3d(
        x=CMesh.observed_positions[:, 0],
        y=CMesh.observed_positions[:, 1],
        z=CMesh.observed_positions[:, 2],
        mode='markers',
        marker=dict(size=2, color=[f'rgb({c[0]},{c[1]},{c[2]})' for c in colors])
    )])
    
    fig.update_layout(
        scene=dict(aspectmode='data'),
        title=f'Confidence: {mask_zero.sum()} red (0), {mask_low.sum()} orange (0-0.25), '
              f'{mask_mid.sum()} yellow (0.25-0.5), {mask_high.sum()} blue (0.5-1), {mask_one.sum()} green (1)'
    )
    
    return fig

## Visualize results

In [ ]:
video_id = 'MPM10'
n_frames = 1618
scene_dir= f'/Experiments/simonep01/ho3d/evaluation/{video_id}'
reader = Ho3dReader(video_dir=scene_dir, root_dir='/Experiments/simonep01/ho3d')
debug_dir= f'debug/{video_id}_{n_frames}'

In [ ]:
eval_dir = f'{debug_dir}/evaluation_results'
with open(f'{eval_dir}/summary.json', 'r') as f:
    summary = json.load(f)

metrics = ['ADI', '3D_IOU']
for key in metrics:
    print(f"{key} =  min:{summary[key]['min']:.4f}   mean:{summary[key]['mean']:.4f}   max:{summary[key]['max']:.4f} ")


In [ ]:
per_frame_metrics = {}
for key in metrics:
    with open(f'{eval_dir}/{key}_per_frame.json', 'r') as f:
        data = json.load(f)
        if key == '3D_IOU':
            per_frame_metrics[key] = [max(val, 60) for val in data]
        else:
            per_frame_metrics[key] = [min(val, 0.01) for val in data]

# Plotting code
fig, axes = plt.subplots(2, 1, figsize=(20, 8))
fig.suptitle('Per-Frame Metrics', fontsize=16, fontweight='bold')

axes = axes.flatten()

for idx, (metric_name, values) in enumerate(per_frame_metrics.items()):
    axes[idx].plot(range(len(values)), values, marker='o', markersize=4, linewidth=2)
    axes[idx].set_xlabel('Frame Number')
    axes[idx].set_ylabel(metric_name)
    axes[idx].set_title(f'{metric_name} over Frames')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{eval_dir}/metrics_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Visualize i=0

In [ ]:
i=0

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

## Visualize i=5

In [ ]:
i=5

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## Visualize i=20

In [ ]:
i=20

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## Visualize i=50

In [ ]:
i=50

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## Visualize i=100

In [ ]:
i=100

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## Visualize i=200

In [ ]:
i=200

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## Visualize i=400

In [ ]:
i=400

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
#visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## Visualize i=800

In [ ]:
i=800

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## visualize i=1000

In [ ]:
i=1000

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## visualize i=1300

In [ ]:
i=1300

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()

## visualize i=1600

In [ ]:
i=1600

In [ ]:
CMesh = MeshWithConfidence.load(f'{debug_dir}/debug/mesh_{i}')
est_pose = np.loadtxt(f'{debug_dir}/ob_in_cam/{reader.id_strs[i]}.txt').reshape(4, 4)
visualize_comparison(CMesh.mesh, est_pose, reader.get_color(i), reader.K)

In [ ]:
CMesh.mesh.show()

In [ ]:
fig = visualize_confidence_pointcloud(CMesh)
fig.show()